## Violation Plots

Notebook for initial visualizations on violation timing distribution for an animal using parsed events history.

In [ ]:
import datajoint as dj
import blob_transformation as bt
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from matplotlib import cm
import pandas as pd
from io_utils import *
import blob_transformation as bt

dj.blob.use_32bit_dims = True
%reload_ext autoreload
%autoreload 2


#### Read in session

In [ ]:
bdata = dj.create_virtual_module('new_acquisition', 'bdatatest')
session_key = {'sessid': 884257}
# session with valid viols: 884257
# session with tup viols: 880698
peh_blob = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)

In [8]:
peh_blob[0].keys()

dict_keys(['peh'])

In [13]:
peh_d = bt.transform_blob(peh_blob[0]['peh'])

ValueError: If using all scalar values, you must pass an index

In [ ]:
# """Multisession read in"""
# subject_session_key = {'ratname': 'R500'}
# peh_blobs = (bdata.ParsedEvents & subject_session_key).fetch("peh", as_dict=True)

In [ ]:
n_sessions = 1
peh = bt.transform_blob(peh_blob[n_sessions-1]['peh'])
n_trials = len(peh)
itrial = 1

In [ ]:
def check_for_spoke_viol(peh_trial_states):
    violation_state = peh_trial_states['violation_state']
    if len(violation_state):
        # wfspoke state only occurs after the go cue
        # and violation only from early spoke if
        # this state doesn't exist
        wf_spoke_state = peh_trial_states['wait_for_spoke']
        if not len(wf_spoke_state):
            return True
    else:
        return False

def get_trial_start_time(peh_trial_states):
  
    # animal can enter cpoke state multiple times
    # before a trial starts and changes the dims
    # one entry = [in_time, out_time]
    # multi entry = [[i_t, o_t], [i_t, o_t]]

    cpoke_state = peh_trial_states['cpoke']
    if cpoke_state.size > 2: 
        # multi entry, grab last entry in time
        trial_start_time = cpoke_state[-1][0]
    else:
        # single entry, grab only in time
        trial_start_time = cpoke_state[0]
    return trial_start_time

def calculate_time_to_viol(peh):

    n_trials = len(peh)
    time_to_viol = []

    for itrial in range(n_trials):
        peh_trial_states = peh[itrial]['states']
        was_early_spoke_viol = check_for_spoke_viol(peh_trial_states)

        if was_early_spoke_viol:
            trial_start_time = get_trial_start_time(peh_trial_states)
            violation_time = peh_trial_states['violation_state'][0]

            time_to_viol.append(violation_time - trial_start_time )
    
    return time_to_viol


In [ ]:
ttv = calculate_time_to_viol(peh)


In [ ]:
plt.hist(ttv)

In [ ]:
peh[itrial]['waves']['go_sound_wave']